In [ ]:
# Depricated from Google Colab as of 2024-12-01

# !sudo apt install python3.9-full python3-pip
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1

# !pip uninstall -y scikit-learn scipy pyparsing -y
# !pip install Cython==0.29.36 scipy==1.9 pyparsing==2.4
# !pip install https://files.pythonhosted.org/packages/04/e2/b43d4205124dd4c1f14606b2e2d78303db993c6653a90bf11dd0ffe23b5b/scikit_learn-0.24.2-cp39-cp39-manylinux2010_x86_64.whl --no-build-isolation


In [1]:
# !pip install pandas==1.5.3
!pip install pandas==2.0.3
!pip install scikit-learn==1.3.2

import pandas as pd
import pickle
import autosklearn
from autosklearn.experimental.askl2 import AutoSklearn2Classifier

import sklearn
from sklearn.model_selection import train_test_split
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np

import autosklearn.classification
import autosklearn.metrics

from smac.optimizer.smbo import SMBO
from smac.runhistory.runhistory import RunInfo, RunValue

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

print(f'pandas version: {pd.__version__}')
print(f'autosklearn version: {autosklearn.__version__}')
print(f'sklearn version: {sklearn.__version__}')


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
pandas version: 2.0.3
autosklearn version: 0.15.0
sklearn version: 1.3.2


In [ ]:
# download data from osf to data/gma_score_prediction
!wget -O osf.zip "https://files.osf.io/v1/resources/gztmd/providers/osfstorage/?zip="
!unzip -o "osf.zip" -d "data/Infant Pose Data/gma_score_prediction"


In [2]:
train_ids = pd.read_csv('./data/Infant Pose Data/gma_score_prediction/train.csv')
val_ids = pd.read_csv('./data/Infant Pose Data/gma_score_prediction/val.csv')
test_ids = pd.read_csv('./data/Infant Pose Data/gma_score_prediction/test.csv')
holdout_ids = pd.read_csv('./data/Infant Pose Data/gma_score_prediction/holdout.csv')
scores = pd.read_csv('./data/Infant Pose Data/gma_score_prediction/gma_score_prediction_scores.csv')

if scores['score'].min() != 0 or not isinstance(scores['score'].min(), int):
    scores = scores[scores['score'].isin([1, 2, 3])]
    scores['score'] = scores['score'].apply(lambda x: int(x) - 1)

In [ ]:
# Create a figure and axis
plt.figure(figsize=(10, 6))

# Create lists of IDs and labels for plotting
id_lists = [train_ids['gma_id'], val_ids['gma_id'], test_ids['gma_id'], holdout_ids['gma_id']]
labels = ['Train', 'Validation', 'Test', 'Holdout']
colors = ['#2ecc71', '#3498db', '#e74c3c', '#f1c40f']

# Plot histograms
plt.hist(id_lists, bins=30, label=labels, color=colors, alpha=0.6)

# Customize plot
plt.title('Distribution of IDs Across Data Splits')
plt.xlabel('GMA ID')
plt.ylabel('Count')
plt.legend()

# Add count annotations
for i, (ids, label, color) in enumerate(zip(id_lists, labels, colors)):
    count = len(ids)
    plt.text(0.02, 0.95 - i*0.05, f'{label}: {count} samples', 
             transform=plt.gca().transAxes, color=color)

plt.show()

# Verify no overlap between sets
print("\nChecking for overlaps between sets:")
print(f"Train-Val overlap: {len(set(train_ids['gma_id']) & set(val_ids['gma_id']))} IDs")
print(f"Train-Test overlap: {len(set(train_ids['gma_id']) & set(test_ids['gma_id']))} IDs")
print(f"Train-Holdout overlap: {len(set(train_ids['gma_id']) & set(holdout_ids['gma_id']))} IDs")
print(f"Val-Test overlap: {len(set(val_ids['gma_id']) & set(test_ids['gma_id']))} IDs")
print(f"Val-Holdout overlap: {len(set(val_ids['gma_id']) & set(holdout_ids['gma_id']))} IDs")
print(f"Test-Holdout overlap: {len(set(test_ids['gma_id']) & set(holdout_ids['gma_id']))} IDs")


In [4]:
## Import features generated from previous notebook
features = pd.read_csv('pose_estimates/gma_score_prediction_pose_estimates/window_mean_by_feature/final/final_window_features.csv')


In [3]:
features_mean = pd.read_csv('pose_estimates/gma_score_prediction_pose_estimates/window_mean_by_feature/final/final_window_features.csv').reset_index(drop=True)
features_mean.drop(columns=['Unnamed: 0'], inplace=True)
features_mean.head()

,infant,Cross-corr_ankle_pos,Cross-corr_elbow_angle,Cross-corr_knee_angle,Cross-corr_wrist_pos,Entropy_ankle_pos,Entropy_elbow_angle,Entropy_knee_angle,Entropy_wrist_pos,IQR_ankle_accel_x,...,Med_ankle_vel_x,Med_ankle_vel_y,Med_elbow_angle_vel,Med_knee_angle_vel,Med_wrist_pos_x,Med_wrist_pos_y,Med_wrist_vel_x,Med_wrist_vel_y,Stdev_elbow_angle,Stdev_knee_angle
0,0,0.405811,0.021501,0.149183,0.255665,0.020787,2.107372,1.888612,0.026190,0.369670,...,0.048133,0.048464,12.586052,6.315284,0.008416,0.103792,0.064117,0.082263,5.928764,2.626044
1,1,0.756703,0.249563,0.746279,0.558858,0.090035,2.894954,3.020745,0.220791,0.830128,...,0.111917,0.269167,21.734614,30.156425,-0.130178,0.439831,0.162296,0.164471,11.666833,10.354837
2,2,0.523569,0.148924,0.196727,0.094564,0.159522,2.602435,2.849426,0.240340,1.350360,...,0.165544,0.246406,22.851980,27.644127,-0.093114,0.280170,0.199028,0.167722,9.847892,9.740183
3,3,0.389591,0.044843,0.286973,0.240591,0.115324,1.685037,2.418695,0.113958,1.847044,...,0.185311,0.152825,8.832697,19.855677,-0.068063,0.337032,0.234152,0.201228,3.541963,6.526309
4,4,-0.040927,-0.045777,-0.015656,0.252009,0.214384,2.790269,2.996049,0.070733,1.420180,...,0.189890,0.218707,21.875280,40.311107,-0.077318,0.201298,0.104210,0.106095,8.457196,13.852110


In [4]:
features_total = pd.read_csv('/workspaces/gma_score_prediction_from_video/pose_estimates/gma_score_prediction_pose_estimates/final_total_features.csv')
features_total.drop(columns=['Unnamed: 0'], inplace=True)
features_total.head()

,infant,Cross-corr_ankle_pos,Cross-corr_elbow_angle,Cross-corr_knee_angle,Cross-corr_wrist_pos,Entropy_ankle_pos,Entropy_elbow_angle,Entropy_knee_angle,Entropy_wrist_pos,IQR_ankle_accel_x,...,Med_ankle_vel_x,Med_ankle_vel_y,Med_elbow_angle_vel,Med_knee_angle_vel,Med_wrist_pos_x,Med_wrist_pos_y,Med_wrist_vel_x,Med_wrist_vel_y,Stdev_elbow_angle,Stdev_knee_angle
0,0,0.223589,-0.217380,0.116477,-0.175872,3.340559,4.236480,3.770771,3.990683,0.208167,...,0.025139,0.034463,5.115890,3.882662,0.060283,0.114144,0.036466,0.044410,3.553537,1.107279
1,1,0.954751,0.908658,0.915769,0.864773,3.740072,4.206915,4.421803,4.040923,0.731753,...,0.093787,0.192321,20.026682,21.996153,-0.151991,0.408578,0.130601,0.137037,4.772821,3.580143
2,2,0.815677,0.790468,0.351167,0.749125,4.463365,4.379137,4.555488,4.603700,0.690399,...,0.090713,0.138048,11.310191,18.580823,-0.001581,0.441662,0.115206,0.118312,4.730931,2.549502
3,3,0.990545,0.460646,0.357742,0.996058,3.972253,3.292102,4.401123,3.933107,0.312466,...,0.042517,0.045916,3.351139,7.965780,0.005877,0.207080,0.033926,0.051585,1.704421,2.301026
4,4,0.110717,-0.244708,0.178843,0.122847,4.266147,4.793099,4.886251,4.099546,1.072120,...,0.136029,0.145235,15.077640,23.935770,-0.063322,0.174741,0.082119,0.080815,3.828461,3.805557


In [ ]:
import seaborn as sns
# plot the correlation matrix for the features in total vs mean 
combined_df = pd.concat([features_mean, features_total], axis=1)

# Calculate the correlation matrix
correlation_matrix = combined_df.corr()

# Extract the part of the correlation matrix that compares 'features_mean' with 'features_total'
# Assuming each data frame has the same column names and order
num_features = len(features_mean.columns)
correlation_submatrix = correlation_matrix.iloc[:num_features, num_features:]

# Visualize the correlation matrix
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_submatrix, annot=False, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Correlation between Features in 'features_mean' and 'features_total'")
plt.xlabel("Features in features_total")
plt.ylabel("Features in features_mean")
plt.show()


In [7]:
fidgety= pd.merge(features_mean, scores[scores['score']==0], on='infant', how='inner').dropna()
absent_fidgety = pd.merge(features_mean, scores[scores['score']==1], on='infant', how='inner').dropna()
atypical_fidgety = pd.merge(features_mean, scores[scores['score']==2], on='infant', how='inner').dropna()

In [5]:
#combine with scores and ages 
features_mean = pd.merge(features_mean, scores, on='infant', how='inner').dropna()
features_total = pd.merge(features_total, scores, on='infant', how='inner').dropna()

print(f'Number of infants with scores: {len(features_mean)}')

Number of infants with scores: 922


In [ ]:
# make scatter plot of features_total vs features_mean for each of the features 
# Create a figure and axis
feature_list = features_mean.columns

n_features = len(feature_list)
n_cols = 3
n_rows = (n_features + n_cols - 1) // n_cols

plt.figure(figsize=(20, 5*n_rows))


for i, feature in enumerate(feature_list):
    print(feature)
    plt.subplot(n_rows, n_cols, i+1)
    plt.scatter(features_mean[feature], features_total[feature])
    plt.legend()

In [ ]:
# plot the distribution of values for each feature as subplots
# Create subplot grid for features
feature_list = features_mean['feature'].unique()

n_features = len(feature_list)
n_cols = 3
n_rows = (n_features + n_cols - 1) // n_cols

#choose colorblind friendly colors for the score
colors = ['#4D4D4D', '#FF0000', '#FFA500']

plt.figure(figsize=(20, 5*n_rows))


# Plot distribution for each feature
for idx, feature in enumerate(feature_list):
    plt.subplot(n_rows, n_cols, idx+1)
    
    plt.scatter(fidgety['age_chronological'], fidgety[feature], c='#8ce8ff')
    plt.scatter(absent_fidgety['age_chronological'], absent_fidgety[feature], c='#ff8c8c')
    plt.scatter(atypical_fidgety['age_chronological'], atypical_fidgety[feature], c='#17b286')
    # plt.scatter(slice['age_chronological'], slice[feature], c=[colors[i] for i in slice['score']], alpha=0.5) 
    plt.title(feature)
    plt.xlim(10, 40)
    plt.xlabel('chronological age')
    plt.ylabel('feature values')

plt.tight_layout()
plt.show()


In [6]:
# Select feature type (total vs windows)
TYPE = 'total'

if TYPE == 'windows':
    print('Run based on windowed features')
    print(f'All infants: {len(features_mean)}')
    features_mean = features_mean[features_mean['score'].isin([0, 1])]
    print(f'Removing infants with GMA Score 3: {len(features_mean)}')
    features_mean = features_mean[(features_mean['age_chronological'] < 50) & (features_mean['age_chronological'] > 10)]
    print(f'Removing infants with ages > 50 weeks or < 10 weeks: {len(features_mean)}')

    holdout_features = features_mean[features_mean['infant'].isin(holdout_ids['gma_id'])]
    train_features = features_mean[features_mean['infant'].isin(train_ids['gma_id']) | features_mean['infant'].isin(val_ids['gma_id'])]
    test_features = features_mean[features_mean['infant'].isin(test_ids['gma_id'])]

elif TYPE == 'total':
    print(f'All infants: {len(features_total)}')
    print('Run based on total features')
    features_total = features_total[features_total['score'].isin([0, 1])]
    print(f'Removing infants with GMA Score 3: {len(features_total)}')
    features_total = features_total[(features_total['age_chronological'] < 50) & (features_total['age_chronological'] > 10)]
    print(f'Removing infants with ages > 50 weeks or < 10 weeks: {len(features_total)}')

    holdout_features = features_total[features_total['infant'].isin(holdout_ids['gma_id'])]
    train_features = features_total[features_total['infant'].isin(train_ids['gma_id']) | features_total['infant'].isin(val_ids['gma_id'])]
    test_features = features_total[features_total['infant'].isin(test_ids['gma_id'])]

X_train = train_features.drop(columns=['infant','age_corrected','age_chronological','score'])
y_train = train_features['score']

X_test = test_features.drop(columns=['infant','age_corrected','age_chronological','score'])
y_test = test_features['score']

X_holdout = holdout_features.drop(columns=['infant','age_corrected','age_chronological','score'])
y_holdout = holdout_features['score']

X_combined = pd.concat([X_train, X_test])
y_combined = pd.concat([y_train, y_test])

All infants: 922
Run based on total features
Removing infants with GMA Score 3: 917
Removing infants with ages > 50 weeks or < 10 weeks: 908


In [ ]:
# Get mean values for each feature per infant
# Create subplot grid for feature means
n_features = len(features_mean.columns)
n_cols = 3
n_rows = (n_features + n_cols - 1) // n_cols

plt.figure(figsize=(20, 5*n_rows))

# Plot distribution for each feature mean
for idx, feature in enumerate(features_mean.columns):
    plt.subplot(n_rows, n_cols, idx+1)
    
    # Get mean and std for this feature
    means = features_mean[feature]
    total = features_total[feature]
    # stds = features_std[feature]
    
    # Plot histogram of means
    plt.hist(means, bins=30, alpha=0.6, color='#3498db')
    plt.hist(total, bins=30, alpha=0.6, color='#2ecc71')

    # set figure title
    plt.suptitle(f'Mean feature values across all infants', fontsize=16, y=1.005)

    plt.title(f'{feature.replace("_", " ")}')
    plt.xlabel('Mean Value')
    plt.ylabel('Count')

plt.tight_layout()
plt.show()



In [ ]:
# plot the distribution of values for each feature

# Get unique features
unique_features = features_mean['feature'].unique()

# Create subplot grid
n_features = len(unique_features)
n_cols = 3
n_rows = (n_features + n_cols - 1) // n_cols

plt.figure(figsize=(20, 5*n_rows))

# Plot distribution for each feature
for idx, feature in enumerate(unique_features):
    plt.subplot(n_rows, n_cols, idx+1)
    
    # Get values for this feature
    feature_vals = features_mean[features_mean['feature'] == feature]['Value']
    plt.suptitle(f'Feature values across all windows', fontsize=16, y=1.005)

    # Plot histogram
    plt.hist(feature_vals, bins=30, alpha=0.6, color='#3498db')
    #scale the y axis logarithmically if needed 
    plt.yscale('log')
    plt.title(feature.replace("_", " "))
    plt.xlabel('Value')
    plt.ylabel('Count')

plt.tight_layout()
plt.show()


In [ ]:
# plot the distribution of scores as discrete values (1, 2, 3)
plt.hist(scores['score'], bins=[0.5, 1.5, 2.5, 3.5], alpha=0.6, color='#3498db')
plt.title('Distribution of scores')
plt.xlabel('Score')
plt.ylabel('Count')
plt.xticks([1, 2, 3])  # Set x-axis ticks to only show 1, 2, 3
plt.show()


In [ ]:
# plot scatter plot of chronological age vs value for each feature and color by score for each infant

for feature in features['feature'].unique():
    plt.scatter(features[features['feature'] == feature]['age_chronological'], features[features['feature'] == feature]['Value'], c=scores['score'], cmap='viridis')
    plt.title(f'{feature.replace("_", " ")}')
    plt.xlabel('Age')
    plt.ylabel('Value')
    plt.show()

In [ ]:
automl = AutoSklearn2Classifier(
    ensemble_size=1,
    dataset_compression=False,
    allow_string_features=False,
    time_left_for_this_task=300,
    per_run_time_limit=30,
    metric=autosklearn.metrics.balanced_accuracy,
    delete_tmp_folder_after_terminate=False,
    memory_limit=None,
    disable_evaluator_output=False,
)

automl.fit(X_train, y_train)
automl.refit(X_train.copy(), y_train.copy())

probabilities = automl.predict_proba(X_test)[:, 1]

filename = f'automl_replication_{TYPE}.pkl'
pickle.dump(automl, open(filename, 'wb')) # save model to disk

In [ ]:
automl.refit(X_train.copy(), y_train.copy())
probabilities = automl.predict_proba(X_test)[:, 1]


In [ ]:
fpr, tpr, _ = roc_curve(y_test, probabilities, pos_label=1)
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, lw=2, alpha=0.3, label=f'ROC AUC = {roc_auc:0.2f}')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Test Set ROC Curve')
plt.legend(loc='best')
plt.show()

In [ ]:
poT = automl.performance_over_time_
poT.plot(
    x="Timestamp",
    kind="line",
    legend=True,
    title="Auto-sklearn accuracy over time",
    grid=True,
)
plt.show()

In [ ]:
# get cross validation scores

cv_scores = automl.cv_results_['mean_test_score']
plt.plot(cv_scores)
plt.title(f'Cross Validation Scores: {(np.median(cv_scores)):.2f}')

In [ ]:
!wget --content-disposition --max-redirect=20 https://osf.io/download/xa43g/
model = pickle.load(open('automl_vanilla_gma_prediction.pkl', 'rb'))

In [ ]:
if TYPE == 'windows':
    automl = pickle.load(open('automl_replication_windows.pkl', 'rb'))
elif TYPE == 'total':
    automl = pickle.load(open('automl_replication_total.pkl', 'rb'))

probabilities = automl.predict_proba(X_holdout)[:, 1]

fpr, tpr, _ = roc_curve(y_holdout, probabilities, pos_label=1)
roc_auc = auc(fpr, tpr)

recall = tpr
precision, _, _ = precision_recall_curve(y_holdout, probabilities)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# ROC Curve
ax1.plot(fpr, tpr, lw=2, alpha=0.3, label=f'ROC AUC = {roc_auc:0.2f}')
ax1.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')  # Add diagonal line
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.set_title(f'Holdout Set ROC Curve: ROC AUC = {roc_auc:0.2f}')
ax1.legend(loc='best')

# Precision-Recall Curve
ax2.plot(recall, precision, lw=2, alpha=0.3, label=f'Average Precision = {average_precision:0.2f}')
ax2.plot([0, 1], [0.2, 0.2], linestyle='--', lw=2, color='navy', label='No Skill')  # No skill baseline
ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')
ax2.set_title(f'Holdout Set Precision-Recall Curve: AUC={average_precision:0.2f}')
ax2.legend(loc='best')

plt.tight_layout()
plt.show()

In [ ]:
print(automl.sprint_statistics())

In [ ]:
# plot feature importance
from sklearn.inspection import plot_partial_dependence, permutation_importance

r = permutation_importance(automl, X_test, y_test, n_repeats=10, random_state=0)
sort_idx = r.importances_mean.argsort()[::-1]

plt.boxplot(
    r.importances[sort_idx].T, labels=[X_train.columns[i] for i in sort_idx]
)

plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

for i in sort_idx[::-1]:
    print(
        f"{X_train.columns[i]:10s}: {r.importances_mean[i]:.3f} +/- "
        f"{r.importances_std[i]:.3f}"
    )

In [17]:
len(mapping)

36

In [18]:
X_test.columns

Index(['Cross-corr_ankle_pos', 'Cross-corr_elbow_angle',
       'Cross-corr_knee_angle', 'Cross-corr_wrist_pos', 'Entropy_ankle_pos',
       'Entropy_elbow_angle', 'Entropy_knee_angle', 'Entropy_wrist_pos',
       'IQR_ankle_accel_x', 'IQR_ankle_accel_y', 'IQR_ankle_pos_x',
       'IQR_ankle_pos_y', 'IQR_ankle_vel_x', 'IQR_ankle_vel_y',
       'IQR_elbow_angle_accel', 'IQR_elbow_angle_vel', 'IQR_knee_angle_accel',
       'IQR_knee_angle_vel', 'IQR_wrist_accel_x', 'IQR_wrist_accel_y',
       'IQR_wrist_pos_x', 'IQR_wrist_pos_y', 'IQR_wrist_vel_x',
       'IQR_wrist_vel_y', 'Mean_elbow_angle', 'Mean_knee_angle',
       'Med_ankle_pos_x', 'Med_ankle_pos_y', 'Med_ankle_vel_x',
       'Med_ankle_vel_y', 'Med_elbow_angle_vel', 'Med_knee_angle_vel',
       'Med_wrist_pos_x', 'Med_wrist_pos_y', 'Med_wrist_vel_x',
       'Med_wrist_vel_y', 'Stdev_elbow_angle', 'Stdev_knee_angle'],
      dtype='object')

In [27]:
# change column names to match the pre-registered model: 

mapping = {
    "Cross-corr_wrist_pos": "Wrist_lrCorr_x",
    "Cross-corr_knee_angle": "Knee_lrCorr_angle",
    "Med_elbow_angle_vel": "Elbow_median_vel_angle",
    "Med_wrist_pos_x": "Wrist_medianx",
    "Med_wrist_pos_y": "Wrist_mediany",
    "Cross-corr_elbow_angle": "Elbow_lrCorr_angle",
    "Med_wrist_angle_vel": "Wrist_median_vel_angle",
    "Med_ankle_pos_x": "Ankle_medianx",
    "Med_ankle_pos_y": "Ankle_mediany",
    "Med_knee_angle_vel": "Knee_median_vel_angle",
    "Entropy_elbow_angle": "Elbow_entropy_angle",
    "Med_wrist_vel_x": "Wrist_medianvelx",
    "IQR_wrist_vel_x": "Wrist_IQRvelx",
    "IQR_ankle_vel_y": "Ankle_IQRvely",
    "IQR_wrist_pos_x": "Wrist_IQRx",
    "IQR_wrist_pos_y": "Wrist_IQRy",
    "IQR_wrist_accel_x": "Wrist_IQRaccx",
    "Mean_elbow_angle": "Elbow_mean_angle",
    "IQR_elbow_angle_vel": "Elbow_IQR_vel_angle",
    "IQR_elbow_angle_accel": "Elbow_IQR_acc_angle", 
    "Cross-corr_knee_angle": "Knee_lrCorr_angle",
    "IQR_ankle_vel_x": "Ankle_IQRvelx",
    "Med_ankle_vel_y": "Ankle_medianvely",
    "Cross-corr_ankle_pos": "Ankle_lrCorr_x",
    "Med_ankle_vel_x": "Ankle_medianvelx",
    "IQR_knee_angle_vel": "Knee_IQR_vel_angle",
    "IQR_ankle_pos_y": "Ankle_IQRy",
    "Mean_knee_angle": "Knee_mean_angle",
    "IQR_knee_angle_accel": "Knee_IQR_acc_angle",
    "Entropy_ankle_pos": "Ankle_meanent",
    "Entropy_wrist_pos": "Wrist_meanent",
    "Stdev_elbow_angle": "Elbow_stdev_angle",
    "IQR_ankle_pos_x": "Ankle_IQRx",
    "Entropy_knee_angle": "Knee_entropy_angle",
    "IQR_wrist_accel_y": "Wrist_IQRaccy",
    "IQR_wrist_vel_y": "Wrist_IQRvely",
    "Stdev_knee_angle": "Knee_stdev_angle",
    "Med_wrist_vel_y": "Wrist_medianvely",
    "IQR_ankle_accel_x": "Ankle_IQRaccx",
    "IQR_ankle_accel_y": "Ankle_IQRaccy"
}

X_test_mapped = X_holdout.rename(columns=mapping)
X_test_mapped.head()

,Ankle_lrCorr_x,Elbow_lrCorr_angle,Knee_lrCorr_angle,Wrist_lrCorr_x,Ankle_meanent,Elbow_entropy_angle,Knee_entropy_angle,Wrist_meanent,Ankle_IQRaccx,Ankle_IQRaccy,...,Ankle_medianvelx,Ankle_medianvely,Elbow_median_vel_angle,Knee_median_vel_angle,Wrist_medianx,Wrist_mediany,Wrist_medianvelx,Wrist_medianvely,Elbow_stdev_angle,Knee_stdev_angle
3,0.110717,-0.244708,0.178843,0.122847,4.266147,4.793099,4.886251,4.099546,1.072120,1.151480,...,0.136029,0.145235,15.077640,23.935770,-0.063322,0.174741,0.082119,0.080815,3.828461,3.805557
15,0.535721,0.881630,0.398479,0.897095,4.006366,4.588035,4.570427,4.354819,0.649356,0.628232,...,0.069975,0.083023,13.390704,11.990214,0.124172,0.296521,0.082656,0.102658,5.838616,3.142516
18,0.745524,-0.167714,0.464058,-0.144914,3.737084,4.269047,3.703178,3.795003,0.534433,0.813965,...,0.063217,0.100243,8.131805,8.785599,-0.036683,-0.164826,0.074119,0.060552,2.384746,1.016005
20,0.275839,0.205690,0.156492,0.246222,4.037414,3.758351,4.576981,3.955645,0.784835,0.789950,...,0.094918,0.103740,6.447228,16.034290,-0.017009,0.546931,0.070477,0.076249,3.820904,2.573926
35,0.601388,0.160124,0.627259,0.497914,3.909577,3.268994,4.578051,3.251786,0.582241,0.686048,...,0.075679,0.095823,3.147127,16.896905,0.009139,0.728190,0.031750,0.026971,0.998561,2.505108


In [8]:
model = pickle.load(open('automl_vanilla_gma_prediction.pkl', 'rb'))

In [24]:
y_test

2      0
39     0
40     0
46     0
57     0
      ..
955    0
958    0
975    0
977    0
984    0
Name: score, Length: 86, dtype: int64

In [28]:
probabilities = model.predict_proba(X_test_mapped)[:, 1]


In [ ]:
fpr, tpr, _ = roc_curve(y_holdout, probabilities, pos_label=1)
roc_auc = auc(fpr, tpr)
precision, recall, _ = precision_recall_curve(y_holdout, probabilities)
average_precision = average_precision_score(y_holdout, probabilities)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# ROC Curve
ax1.plot(fpr, tpr, lw=2, alpha=0.3, label=f'ROC AUC = {roc_auc:0.2f}')
ax1.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')  # Add diagonal line
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.set_title(f'Holdout Set ROC Curve: ROC AUC = {roc_auc:0.2f}')
ax1.legend(loc='best')

ax2.plot(recall, precision, lw=2, alpha=0.3, label=f'Average Precision = {average_precision:.2f}')
ax2.plot([0, 1], [0.2, 0.2], linestyle='--', lw=2, color='navy', label='No Skill')  # No skill baseline
ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')
ax2.set_title(f'Holdout Set Precision-Recall Curve: AUC={average_precision:0.2f}')
ax2.legend(loc='best')

plt.tight_layout()
plt.show()